In [ ]:
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
from mrjob.job import MRJob
import numpy as np

In [ ]:
#WordCount
class MRWordCount(MRJob):
  def mapper(self, _, line):
    for word in line.split():
      yield (word, 1)

  def reducer(self, word, counts):
    yield (word, sum(counts))

In [ ]:
#Average Temperatures
class AvgTemperature(MRJob):
  def mapper(self, _, line):
    month, temperature = line.split()
    yield month, (int(temperature), 1)

  def _reducer_combiner(self, month, temperatures):
    sum, count = 0, 0
    for tmp, c in temperatures:
      sum = sum + tmp
      count = count + c
    avg = sum / count
    return month, (avg, count)

    def combiner(self, month, temperatures):
      yield self._reducer_combiner(month, temperatures)

    def reducer(self, month, temperatures):
      month, (avg, count) = self._reducer_combiner(month, temperatures)
      yield month, avg

In [ ]:
#KMeans
class ClusterAssignmentMapReduce(MRJob):
  def configure_args(self):
    super(ClusterAssignmentMapReduce, self).configure_args()
    self.add_file_arg("--clusters", help="File containing cluster centroids")

  def mapper_init(self):
    with open(self.options.clusters, 'r') as f:
      self.cluster_centroids = [float(line.strip()) for line in f]

  def mapper(self, _, line):
    data_point = float(line.strip())
    nearest_cluster = min(self.cluster_centroids, key = lambda centroid: abs(centroid - data_point))
    yield nearest_cluster, data_point

  def reducer(self, cluster, data_points):
    yield cluster, list(data_points)